# This notebook shows how to renew an existing slice

## Configure the environment

In [23]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTGZDCHE4TGZBSG5RTKMRYGE2WCMDGGJSGIYTGGE4DQOLGHBRWCMR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRSGA2DGOJVGM3DMMZGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXWGMZYG4YTAMLGHE2DOMBTGQYTIZLFHE3DGMRWGUZDKZBTHEYGMP3UPFYGKPLSMVTHEZLTNBKG623FNYTHI4Z5GE3DEMRQGA2DIMBWGU4DKJTWMVZHG2LPNY6XMMROGATGY2LGMV2GS3LFHU4DMNBQGAYDAMA


## Create slice manager object

In [24]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host,refresh_token=fabric_refresh_token, project_name='all', scope='all')

## Refresh the ID Token

In [25]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print("New Refresh Token: {}".format(fabric_refresh_token))
print("Stored new Refresh Token")
%store fabric_refresh_token

New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTONJVG5SGKNJSMVRDONDBMRTDQYZXGQYDQNZRMRSGCZJUGVSWIMJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRSGA2DINRWGU4DMNRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
Stored new Refresh Token
Stored 'fabric_refresh_token' (str)


## Query existing Slices

In [26]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "27a459ba-15ad-46ea-8f28-bf1102a48884",
    "slice_id": "b131992d-7fc1-42c3-b77e-6fb0e4f2baa5",
    "slice_name": "ToBeRemoved",
    "slice_state": "StableOK"
}, {
    "graph_id": "f6b7ff90-0c37-4457-95c3-de91c92d6810",
    "slice_id": "31370a62-71b4-42ee-9f3a-0799ffb0aa22",
    "slice_name": "ExampleSlice",
    "slice_state": "Configuring"
}]


## Retrieve Slice ID

In [27]:
#Name of slice to be renewed
TargetSliceName = "ExampleSlice"

#Search for Slice ID from Slice Name
slice_id = list(filter(lambda slices: slices.slice_name == TargetSliceName, slices))[0].slice_id

print(slice_id)

31370a62-71b4-42ee-9f3a-0799ffb0aa22


## Renew Slice

In [30]:
import datetime

#Slice renew to end in 1 day
end_date = (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%Y-%m-%d %H:%M:%S")

status, result = slice_manager.renew(slice_id=slice_id,new_lease_end_time = end_date)

print("Response Status {}".format(result))

Response Status (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.19.8', 'Date': 'Wed, 26 May 2021 15:59:36 GMT', 'Content-Type': 'application/json', 'Content-Length': '102', 'Connection': 'keep-alive'})
HTTP response body: "Unable to renew Slice# 31370a62-71b4-42ee-9f3a-0799ffb0aa22 that is not yet stable, try again later"


